In [2]:
from PIL import Image
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage import data, img_as_float
import binascii
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

import numpy as np
import imutils



# Step 1: Biometric using Face Geometry

In [3]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

originalImage = cv2.imread('face2.png')
img = cv2.resize(originalImage, (300,300))
face = []
leftEyes = []
rightEyes = []

def detect_faces(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = cascade.detectMultiScale(gray_frame, 1.3, 5)
    face.append(coords[0][0])
    face.append(coords[0][1])

    print("face",coords)

    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return None
    for (x, y, w, h) in biggest:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)    

        frame = img[y:y + h, x:x + w]
    return frame


def detect_eyes(img, cascade):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = cascade.detectMultiScale(gray_frame, 1.3, 5)  # detect eyes
    print("eyes" , eyes)
    leftEyes.append(eyes[0][0])
    leftEyes.append(eyes[0][1])
    rightEyes.append(eyes[1][0])
    rightEyes.append(eyes[1][1])

    width = np.size(img, 1)  # get face frame width
    height = np.size(img, 0)  # get face frame height
    left_eye = None
    right_eye = None
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye center
        if eyecenter < width * 0.5:
            left_eye = img[y:y + h, x:x + w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)
        else:
            right_eye = img[y:y + h, x:x + w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,225,255),2)

    return left_eye, right_eye


face_frame = detect_faces(img, face_cascade)
if face_frame is not None:
    eyes = detect_eyes(face_frame, eye_cascade)

    
cv2.imshow('my image',img)
cv2.waitKey(0)
cv2.destroyAllWindows() 



face [[ 82  48 178 178]]
eyes [[95 42 43 43]
 [32 40 49 49]]


In [5]:
print("face", face)
print("leftEyes", leftEyes)
print("rightEyes", rightEyes)

face [82, 48]
leftEyes [95, 42]
rightEyes [32, 40]


# Key Generation

In [6]:
distance = np.sqrt( (leftEyes[0]-rightEyes[0])**2 + (leftEyes[1]-rightEyes[1])**2 )
distance

key = int(str(distance)[:2])
key

63

# Step 2: Encrypt msg using Transposition Cipher

In [7]:
Number = key
Sum = 0
while(Number > 0):
    r = Number % 10
    Sum = Sum + r
    Number = Number//10
print(Sum)
face_key = Sum


9


In [8]:
#Transposition cipher 

def encryptMessage(key, message):
    ciphertext = [''] * key
   
    for col in range(key):
        position = col
        while position < len(message):
            ciphertext[col] += message[position]
            position += key
    return ''.join(ciphertext) #Cipher text
    
myMessage = input("Enter data to be encoded(Without Space) : ")
#myMessage = "Transposition Cipher"

ciphertext = encryptMessage(face_key, myMessage)
   
print("Cipher Text is")
print(ciphertext + '|')
#pyperclip.copy(ciphertext)

Enter data to be encoded(Without Space) : ABCabc12
Cipher Text is
ABCabc12|


# lsb for alphanumeric

In [23]:
import cv2
#import docopt
import numpy as np


class SteganographyException(Exception):
    pass


class LSBSteg():
    def __init__(self, im):
        self.image = im
        self.height, self.width, self.nbchannels = im.shape
        self.size = self.width * self.height
        
        self.maskONEValues = [1,2,4,8,16,32,64,128]
        #Mask used to put one ex:1->00000001, 2->00000010 .. associated with OR bitwise
        self.maskONE = self.maskONEValues.pop(0) #Will be used to do bitwise operations
        
        self.maskZEROValues = [254,253,251,247,239,223,191,127]
        #Mak used to put zero ex:254->11111110, 253->11111101 .. associated with AND bitwise
        self.maskZERO = self.maskZEROValues.pop(0)
        
        self.curwidth = key  # Current width position
        self.curheight = key # Current height position
        self.curchan = 0   # Current channel position

    def put_binary_value(self, bits): #Put the bits in the image
        for c in bits:
            val = list(self.image[self.curheight,self.curwidth]) #Get the pixel value as a list
            if int(c) == 1:
                val[self.curchan] = int(val[self.curchan]) | self.maskONE #OR with maskONE
            else:
                val[self.curchan] = int(val[self.curchan]) & self.maskZERO #AND with maskZERO
                
            self.image[self.curheight,self.curwidth] = tuple(val)
            self.next_slot() #Move "cursor" to the next space
        
    def next_slot(self):#Move to the next slot were information can be taken or put
        if self.curchan == self.nbchannels-1: #Next Space is the following channel
            self.curchan = 0
            if self.curwidth == self.width-1: #Or the first channel of the next pixel of the same line
                self.curwidth = 0
                if self.curheight == self.height-1:#Or the first channel of the first pixel of the next line
                    self.curheight = 0
                    if self.maskONE == 128: #Mask 1000000, so the last mask
                        raise SteganographyException("No available slot remaining (image filled)")
                    else: #Or instead of using the first bit start using the second and so on..
                        self.maskONE = self.maskONEValues.pop(0)
                        self.maskZERO = self.maskZEROValues.pop(0)
                else:
                    self.curheight +=1
            else:
                self.curwidth +=1
        else:
            self.curchan +=1

    def read_bit(self): #Read a single bit int the image
        val = self.image[self.curheight,self.curwidth][self.curchan]
        val = int(val) & self.maskONE
        self.next_slot()
        if val > 0:
            return "1"
        else:
            return "0"
    
    def read_byte(self):
        return self.read_bits(8)
    
    def read_bits(self, nb): #Read the given number of bits
        bits = ""
        for i in range(nb):
            bits += self.read_bit()
        return bits

    def byteValue(self, val):
        return self.binary_value(val, 8)
        
    def binary_value(self, val, bitsize): #Return the binary value of an int as a byte
        binval = bin(val)[2:]
        if len(binval) > bitsize:
            raise SteganographyException("binary value larger than the expected size")
        while len(binval) < bitsize:
            binval = "0"+binval
        return binval

    def encode_text(self, txt):
        l = len(txt)
        binl = self.binary_value(l, 16) #Length coded on 2 bytes so the text size can be up to 65536 bytes long
        self.put_binary_value(binl) #Put text length coded on 4 bytes
        for char in txt: #And put all the chars
            c = ord(char)
            self.put_binary_value(self.byteValue(c))
        return self.image
       
    def decode_text(self):
        ls = self.read_bits(16) #Read the text size in bytes
        l = int(ls,2)
        i = 0
        unhideTxt = ""
        while i < l: #Read all bytes of the text
            tmp = self.read_byte() #So one byte
            i += 1
            unhideTxt += chr(int(tmp,2)) #Every chars concatenated to str
        return unhideTxt

    def encode_image(self, imtohide):
        w = imtohide.width
        h = imtohide.height
        if self.width*self.height*self.nbchannels < w*h*imtohide.channels:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        binw = self.binary_value(w, 16) #Width coded on to byte so width up to 65536
        binh = self.binary_value(h, 16)
        self.put_binary_value(binw) #Put width
        self.put_binary_value(binh) #Put height
        for h in range(imtohide.height): #Iterate the hole image to put every pixel values
            for w in range(imtohide.width):
                for chan in range(imtohide.channels):
                    val = imtohide[h,w][chan]
                    self.put_binary_value(self.byteValue(int(val)))
        return self.image

                    
    def decode_image(self):
        width = int(self.read_bits(16),2) #Read 16bits and convert it in int
        height = int(self.read_bits(16),2)
        unhideimg = np.zeros((width,height, 3), np.uint8) #Create an image in which we will put all the pixels read
        for h in range(height):
            for w in range(width):
                for chan in range(unhideimg.channels):
                    val = list(unhideimg[h,w])
                    val[chan] = int(self.read_byte(),2) #Read the value
                    unhideimg[h,w] = tuple(val)
        return unhideimg
    
    def encode_binary(self, data):
        l = len(data)
        if self.width*self.height*self.nbchannels < l+64:
            raise SteganographyException("Carrier image not big enough to hold all the datas to steganography")
        self.put_binary_value(self.binary_value(l, 64))
        for byte in data:
            byte = byte if isinstance(byte, int) else ord(byte) # Compat py2/py3
            self.put_binary_value(self.byteValue(byte))
        return self.image

    def decode_binary(self):
        l = int(self.read_bits(64), 2)
        output = b""
        for i in range(l):
            output += chr(int(self.read_byte(),2)).encode("utf-8")
        return output



In [24]:
res= str(ciphertext)
print("res",res)
#encoding
steg = LSBSteg(cv2.imread("source_img.png"))
img_encoded = steg.encode_text(res)
cv2.imwrite("my_new_image.png", img_encoded)

#decoding
im = cv2.imread("my_new_image.png")
steg = LSBSteg(im)
print("Text value:",steg.decode_text())

res ABCabc12
Text value: ABCabc12


# # Step 3: Image Steganography using LSB


In [16]:
'''
- First carrier image or cover image has been read and converted in to array of bits
- the secret message which is in the form of characters are converted in to the “ASCII” values
- ASCII values are converted in to array of bits
- key will gives unique pixel position of cover image to embed the data LSB of particular image pixel of cover object

Output: Stego-image has been generated which contains secret message embedded within cover image.

'''
#res = input("Enter data to be encoded(Without Space) : ") 
res= str(ciphertext)
print("res",res)

data = ''.join(format(ord(i), 'b') for i in res) 
#print("The string after binary conversion : " + data) 
msg_len = len(data)
print("msg length",msg_len)
            
    
i=0

with Image.open("source_img.png") as img:
   
    width, height = img.size
    print(width)
    print(height)
    #img.show()
    print(key)
    for x in range(key, width):
        for y in range(key, height):
            pixel = list(img.getpixel((x, y)))
            for n in range(0,3):
                if(i < len(data)):
                    pixel[n] = pixel[n] & ~1 | int(data[i])
                    i+=1
                    
            img.putpixel((x,y), tuple(pixel))
            
    
    img.save("source_secret.png", "PNG")
        
        

res ABCabc12
msg length 54
4000
2249
63


# Step 4: Data Extraction using LSB

In [11]:
'''
- Embedded secret data has extracted from stego-image
- Extracting secret data the hand key is used

'''
extracted_bin = []
with Image.open("source_secret.png") as img:
    width, height = img.size
    byte = []
    for x in range(key, width):
        for y in range(key, height):
            pixel = list(img.getpixel((x, y)))
            for n in range(0,3):
                extracted_bin.append(pixel[n]&1)

data = "".join([str(x) for x in extracted_bin])
msg =''

for i in range(msg_len):
    msg+=str(extracted_bin[i])
#print(msg)

decoded_msg = ''.join(chr(int(msg[i*7:i*7+7],2)) for i in range(len(msg)//7))
print("decoded_msg:= ",decoded_msg)


decoded_msg:=  ghjedf


# Step 5: decrypt msg using Transposition Cipher

In [12]:
import math
def decryptMessage(key, message):
    
    numOfColumns = math.ceil(len(message) / key)
    numOfRows = key
    numOfShadedBoxes = (numOfColumns * numOfRows) - len(message)
    plaintext = [""]  * numOfColumns
    col = 0
    row = 0
   
    for symbol in message:
        plaintext[col] += symbol
        col += 1
        if (col == numOfColumns) or (col == numOfColumns - 1 and row >= numOfRows - numOfShadedBoxes):
                col = 0 
                row += 1 
    return ''.join(plaintext)
            
plaintext = decryptMessage(face_key, decoded_msg)
   
print("The plain text is")
print(plaintext)


The plain text is
ghjedf
